##  네이버에서 크롤링한 영화 리뷰 데이터의 긍/부정 분석

### 네이버 영화 리뷰 데이터
- 리뷰 작성자가 준 평점이 0.5보다 크면 1로, 이하면 0으로 라벨링해놓은 데이터
- 한글 데이터이므로 영문 전처리와는 다르게 진행

In [1]:
import numpy as np
import pandas as pd
import os
import matplotlib.pyplot as plt
import seaborn as sns

### 데이터 전처리
- 불용어 제거 (ㄱ-ㅎ, ㅏ-ㅜ 등 한글 자모음이 아닌 나머지는 제외)
    - 정규식에서 한글 글자 표현 : 가-힣
    - 불용어는 파일을 이용하는 것이 좋음
- 리뷰를 토큰화
    - 학습/테스트 데이터 numpy 파일 저장
    - 토큰화 시 생성된 어휘사전 json 파일로 저장
- 한글 토큰화 패키지
    - konlp 패키지 사용

In [2]:
train_data = pd.read_csv('./data/ratings_train.txt',header=0, delimiter='\t', 
                         quoting=3)
train_data.head()

,id,document,label
0,9976970,아 더빙.. 진짜 짜증나네요 목소리,0
1,3819312,흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나,1
2,10265843,너무재밓었다그래서보는것을추천한다,0
3,9045019,교도소 이야기구먼 ..솔직히 재미는 없다..평점 조정,0
4,6483659,사이몬페그의 익살스런 연기가 돋보였던 영화!스파이더맨에서 늙어보이기만 했던 커스틴 ...,1


In [3]:
print('학습데이터 전체 개수 : ', len(train_data))

학습데이터 전체 개수 :  150002


In [4]:
# 리뷰 문자 길이 확인
train_length = train_data['document'].astype(str).apply(len)
train_length.head()

0    19
1    33
2    17
3    29
4    61
Name: document, dtype: int64

In [5]:
# 리뷰 통계정보
print('리뷰 길이 최댓값 : {}'.format(np.max(train_length)))
print('리뷰 길이 최솟값 : {}'.format(np.min(train_length)))
print('리뷰 길이 평균값 : {:.2f}'.format(np.mean(train_length)))
print('리뷰 길이 표준편차 : {:.2f}'.format(np.std(train_length)))
print('리뷰 길이 중간값 : {}'.format(np.median(train_length)))
print('리뷰 길이 재1사분위 : {}'.format(np.percentile(train_length, 25)))
print('리뷰 길이 제3사분위 : {}'.format(np.percentile(train_length, 75)))

리뷰 길이 최댓값 : 158
리뷰 길이 최솟값 : 1
리뷰 길이 평균값 : 35.24
리뷰 길이 표준편차 : 29.58
리뷰 길이 중간값 : 27.0
리뷰 길이 재1사분위 : 16.0
리뷰 길이 제3사분위 : 42.0


- 리뷰의 평균 길이 : 35.24
- 중간값과 차이가 있고, 편차가 29 정도인 것을 보아 리뷰 길이 범위 차이가 많이 남

In [6]:
train_data['label'].value_counts()

0    75173
1    74829
Name: label, dtype: int64

In [7]:
# 긍/부정 리뷰 수 확인
print('리뷰1(긍정)', train_data['label'].value_counts()[1])
print('리뷰0(부정)', train_data['label'].value_counts()[0])

리뷰1(긍정) 74829
리뷰0(부정) 75173


### 긍/부정 분석을 위한 자연어 처리
- 문자열 데이터를 수치화하기 위한 작업

### 데이터 전처리
- 5단계로 진행

1. 정규화로 한국어만 남기기
2. 형태소 분석기로 어간 추출하기
    - 토큰화
3. 불용어 제거하기
4. 문자를 인덱스 벡터로 전환하기
5. 패딩 처리하기


- 한국어 텍스트를 전처리할 때는 konlpy의 okt()를 사용하여 형태소 분석
    - okt()는 트위터사에서 제작한 한글 형태소 분석기

In [8]:
!pip install konlpy

  Using cached konlpy-0.6.0-py2.py3-none-any.whl (19.4 MB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 465.6/465.6 kB 33.3 MB/s eta 0:00:00


In [9]:
import re  # 정규식 적용해주는 패키지
import json  # 어휘사전 저장 및 load

# 한글 형태소 분석기 okt
# open korea text (트위터에서 제작 - 현재 다른 기관에서 프로젝트 진행)
import konlpy
from konlpy.tag import Okt

In [10]:
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import Tokenizer

In [11]:
train_data['document'][:5]

0                                  아 더빙.. 진짜 짜증나네요 목소리
1                    흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나
2                                    너무재밓었다그래서보는것을추천한다
3                        교도소 이야기구먼 ..솔직히 재미는 없다..평점 조정
4    사이몬페그의 익살스런 연기가 돋보였던 영화!스파이더맨에서 늙어보이기만 했던 커스틴 ...
Name: document, dtype: object

In [12]:
# 1. 한글과 공백 제외 모두 제거
import re

# re.sub() : 어떤 패턴을 특정 문자열로 대체
# 모든 한글글자, 한글 자/모음을 제외한 나머지 글자는 공백으로 대체
# 제외 정규식 '[^정규식패턴]'
re.sub('[^가-힣 ㄱ-ㅎ ㅏ-ㅣ\\s ]',' ','가나다 abcde ^%$##$ 1234 뮤직')

'가나다                   뮤직'

- 한글이 아닌 영문과 특수문자는 공백으로 대체됨

In [13]:
review = '교도소 이야기구먼 ..솔직히 재미는 없다..평점 조정'
review_text = re.sub('[^가-힣 ㄱ-ㅎ ㅏ-ㅣ\\s ]',' ',review)
review_text

'교도소 이야기구먼   솔직히 재미는 없다  평점 조정'

### 한글 형태소 단위 토크나이징
- KoNLPy 사용
    - KoNLPy에서는 여러 형태소 분석기를 제공하며, 각 형태소 분석기별로 분석한 결과는 다를 수 있다.
    - 각 형태소 분석기는 클래스 형태로 되어 있고 이를 객체로 생성한 후 매서드를 호출해서 토크나이징할 수 있다.

- 형태소 분석 및 품사 태깅
    - 형태소란 의미를 가지는 가장 작은 단위로서 더 쪼개지면 의미를 상실하는 것들을 말한다.
    - 형태소 분석이란 의미를 가지는 단위를 기준으로 문장을 살펴보는 것을 의미한다.
    - KoNLPy는 기존에 C, C++, Java 등의 언어를 통해 형태소 분석을 할 수 있는 좋은 라이브러리들을 파이썬 라이브러리로 통합해서 사용할 수 있록 하여 한국어 구문 분석을 쉽게 할 수 있도록 만들어진 라이브러리이다.
    - KoNLPy에는 다양한 형태소 분석기들이 객체 형태로 포함돼 있으며 다음과 같은 각 형태소 분석기 목록이 있다.
        - Hannanum
        - Kkma
        - Komoran
        - Mecab
        - Okt(Twitter)
        - 모두 동일한 형태소 분석 기능 제공하지만 각기 성능이 조금씩 다름

- Okt 사용
    1. 객체 생성
        - Okt() 생성자 함수 사용
    2. 필요 함수 사용
        - Okt.morphs() 텍스트를 형태소 단위로 나눈다.
        - 파라미터로 norm, stem 사용
            - norm : 문장 정규화 여부
            - stem : 어간 추출 여부
            - 기본값 : False
        - Okt.nouns() 텍스트에서 명사만 추출
        - Okt.phrases() 텍스트에서 어절 추출
        - Okt.pos() 각 품사를 태깅
            - 태깅 : 주어진 텍스트를 형태소 단위로 나누는 것을 의미
                - 나누어진 형태소와 해당 품사와 함께 리스트화 하는 것

In [14]:
# 2. 형태소 분석 
from konlpy.tag import Okt

okt = Okt()  # 객체 생성
word_review = okt.morphs(review_text, stem=True)
word_review

['교도소', '이야기', '구먼', '솔직하다', '재미', '는', '없다', '평점', '조정']

### okt 에러 발생 시 java 설치
- No JVM shared library file (jvm.dll) found. Try setting up the JAVA_HOME environment variable properly.


- 오라클 홈페이지에서 다운로드
- 환경변수에 java 패스 등록

In [15]:
# 3. stop_words 제거 (불용어, 관용어구)
stop_words = ['은','는','이','가','하','아','것','들','의','있','되','수','보','주','등','한','구먼']
lis = [token for token in word_review if not token in stop_words]
lis

['교도소', '이야기', '솔직하다', '재미', '없다', '평점', '조정']

In [16]:
# 4. Tokenizer() 객체를 활용해서 단어와 수자 매핑
tokenizer = Tokenizer()
tokenizer.fit_on_texts(lis)  # 토큰을 유니크하게 정리해서 각 단어들에 대해 숫자 매핑
train_sequences = tokenizer.texts_to_sequences(lis)  # n행 ㅂ열로 변환
train_sequences = np.reshape(train_sequences,(1,-1))  # 1행 n열로 변경

In [17]:
# 4-1. 어휘사전 확인 및 저장
word_vocab = tokenizer.word_index
word_vocab

{'교도소': 1, '이야기': 2, '솔직하다': 3, '재미': 4, '없다': 5, '평점': 6, '조정': 7}

In [18]:
# 5. 패딩 작업해서 모든 리뷰의 문장 길이를 동일하게 작업
MAX_SEQUENCE_LENGTH = 10  # 문장 길이 (상수이므로 대문자)

# padding='post' : 문장의 뒷부분을 padding

# 패딩 처리
train_inputs = pad_sequences(train_sequences, maxlen=MAX_SEQUENCE_LENGTH,
                            padding='post')
train_inputs

array([[1, 2, 3, 4, 5, 6, 7, 0, 0, 0]], dtype=int32)

In [19]:
# 전처리 함수 만들기
def preprocessing(review, okt, remove_stopwords = False, stop_words =[]):
    #함수인자설명
    # review: 전처리할 텍스트
    # okt: okt객체를 반복적으로 생성하지 않고 미리 생성 후 인자로 받음
    # remove_stopword: 불용어를 제거할지 여부 선택. 기본값 False
    # stop_words: 불용어 사전은 사용자가 직접 입력, 기본값 빈 리스트


    # 1. 한글 및 공백 제외한 문자 모두 제거
    review_text = re.sub('[^가-힣ㄱ-ㅎㅏ-ㅣ\\s]','',review)
    # review에서 한글및공백을 제외한 문자는 ' '로 치환


    # 2. okt 객체를 활용하여 형태소 단어로 나눔
    word_review = okt.morphs(review_text,stem=True)


    if remove_stopwords:
        # 3. 불용어 제거
        word_review = [token for token in word_review if not token in stop_words]
    return word_review

In [20]:
# 전체 학습 데이터에 대해 전처리
okt = Okt()  # 형태소 분석기 객체 생성
clean_train_review = []  # 한글 전처리 후 형태소 분석된 리뷰를 저장
stop_words=['은','는','이','가','하','아','것','들','의','있','되','수','보','주','등','한','구먼']

In [21]:
train_data.shape

(150002, 3)

In [22]:
# 시간상 일부 데이터만 사용
for review in train_data['document'][:10000] :  # 10000개만 가져와서 하기
    if type(review) == str : 
        clean_train_review.append(preprocessing(review,okt,remove_stopwords = True, stop_words =stop_words))
    else : 
        clean_train_review.append([])  # 리뷰가 문자열이 아니면 빈칸으로 놔두기
clean_train_review[:4]     

[['더빙', '진짜', '짜증나다', '목소리'],
 ['흠', '포스터', '보고', '초딩', '영화', '줄', '오버', '연기', '조차', '가볍다', '않다'],
 ['너', '무재', '밓었', '다그', '래서', '보다', '추천', '다'],
 ['교도소', '이야기', '솔직하다', '재미', '없다', '평점', '조정']]

In [23]:
# 테스트 리뷰도 동일하게 전처리
test_data = pd.read_csv('./data/ratings_test.txt', header=0,
                        delimiter='\t', quoting=3)

In [24]:
clean_test_review=[]
for review in test_data['document'][:10000] :  # 10000개만 가져와서 하기
    if type(review) == str : 
        clean_test_review.append(preprocessing(review,okt,remove_stopwords = True, stop_words =stop_words))
    else : 
        clean_test_review.append([]) # 리뷰가 문자열이 아니면 빈칸으로 놔두기
clean_test_review[:4]     

[['굳다', 'ㅋ'],
 [],
 ['뭐', '야', '평점', '나쁘다', '않다', '점', '짜다', '리', '더', '더욱', '아니다'],
 ['지루하다', '않다', '완전', '막장', '임', '돈', '주다', '보기', '에는']]

3. 문자 리뷰를 벡터 데이터로 변환
- 라벨 데이터는 긍/부정, 1/0으로 처리되어 있음

In [25]:
# 학습데이터로만 진행

tokenizer = Tokenizer()
tokenizer.fit_on_texts(clean_train_review)  # 어휘사전 생성

In [26]:
# 학습데이터로 생성된 어휘사전을 포함하고 있는 tokenizer
train_sequences = tokenizer.texts_to_sequences(clean_train_review)
test_sequences = tokenizer.texts_to_sequences(clean_test_review)

In [27]:
word_vocab = tokenizer.word_index
MAX_SEQUENCE_LENGTH = 8

In [28]:
# 학습 데이터 패딩 및 np 데이터 변환
train_inputs = pad_sequences(train_sequences,maxlen=MAX_SEQUENCE_LENGTH,
                            padding='post')
train_labels = np.array(train_data['label'][:10000])
train_inputs[1]

array([ 640,    1,  168, 1827,   29,  888,  728,   25], dtype=int32)

In [29]:
train_inputs.shape

(10000, 8)

In [30]:
train_labels.shape

# 둘 다 10000개로 개수 동일한지 확인

(10000,)

In [31]:
# 테스트 데이터 패딩 및 np 데이터 변환
test_inputs = pad_sequences(test_sequences,maxlen=MAX_SEQUENCE_LENGTH,
                            padding='post')
test_labels = np.array(test_data['label'][:10000])
test_inputs[100]

array([656, 339, 112, 357,  16, 862, 114,   0], dtype=int32)

In [32]:
test_inputs.shape

(10000, 8)

In [33]:
test_labels.shape

# 둘 다 개수 10000개로 동일한지 확인

(10000,)

### 4. 전처리 완료된 데이터 (형태소 분석, 토큰화) numpy 파일로 저장
- 어휘사전도 json 파일로 저장

In [34]:
DATA_PATH = 'cleandata/'  # .npy 파일 저장 경로지정, # 없으면 생성

TRAIN_INPUT_DATA = 'nsmc_train_input.npy'
TRAIN_LABEL_DATA = 'nsmc_train_label.npy'

TEST_INPUT_DATA = 'nsmc_test_input.npy'
TEST_LABEL_DATA = 'nsmc_test_label.npy'

DATA_CONFIGS = 'data_configs.json'  # 어휘사전

In [35]:
data_configs={}
data_configs['vocab']=word_vocab  # 어휘사전
data_configs['vocab_size']=len(word_vocab)+1  # 특수 vocab 0 추가

In [36]:
# 데이터를 파일로 저장
import os
if not os.path.exists(DATA_PATH) :  # 해당 경로가 없으면
    os.makedirs(DATA_PATH)  # 생성
    
# 학습데이터 저장
np.save(open(DATA_PATH+TRAIN_INPUT_DATA,'wb'),train_inputs)
np.save(open(DATA_PATH+TRAIN_LABEL_DATA,'wb'),train_labels)

# 테스트데이터 저장
np.save(open(DATA_PATH+TEST_INPUT_DATA,'wb'),test_inputs)
np.save(open(DATA_PATH+TEST_LABEL_DATA,'wb'),test_labels)

# 단어사전 json으로 저장
json.dump(data_configs,open(DATA_PATH+DATA_CONFIGS,'w'),ensure_ascii=False)

### 단어사전 (어휘사전)
- 단어와 매칭되는 임의의 숫자가 key:value 형태로 저장된 파일

### 전처리 완료된 학습/테스트 데이터 활용 모델링 진행

In [37]:
import tensorflow as tf
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras import layers

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import json
from tqdm import tqdm

In [38]:
# 1. 데이터 불러오기
DATA_PATH = 'cleandata/'  # .npy파일 저장 경로지정, 없으면 생성
TRAIN_INPUT_DATA = 'nsmc_train_input.npy'
TRAIN_LABEL_DATA = 'nsmc_train_label.npy'
TEST_INPUT_DATA = 'nsmc_test_input.npy'
TEST_LABEL_DATA = 'nsmc_test_label.npy'
DATA_CONFIGS = 'data_configs.json' #어휘사전
DATA_OUT = 'data_out/'

In [39]:
train_labels[0]

0

In [40]:
DATA_PATH+TRAIN_INPUT_DATA

'cleandata/nsmc_train_input.npy'

In [41]:
train_input = np.load(open(DATA_PATH+TRAIN_INPUT_DATA,'rb'))
train_input[0]

array([682,  20, 241, 776,   0,   0,   0,   0], dtype=int32)

In [42]:
train_label = np.load(open(DATA_PATH + TRAIN_LABEL_DATA,'rb'))
prepro_configs = json.load(open(DATA_PATH + DATA_CONFIGS,'r'))

In [43]:
train_label.shape

(10000,)

In [44]:
train_input.shape

(10000, 8)

In [45]:
train_input[1]

array([ 640,    1,  168, 1827,   29,  888,  728,   25], dtype=int32)

In [46]:
# 숫자에 대응되는 단어 확인하기 위해 prepro_configs의 key, value 뒤집기
tmp_dict = dict(map(reversed,prepro_configs['vocab'].items()))
tmp_dict

{1: '영화',
 2: '하다',
 3: '보다',
 4: '에',
 5: '을',
 6: '도',
 7: '를',
 8: '없다',
 9: '이다',
 10: '있다',
 11: '좋다',
 12: '다',
 13: '너무',
 14: '정말',
 15: '재밌다',
 16: '만',
 17: '적',
 18: '같다',
 19: '되다',
 20: '진짜',
 21: '아니다',
 22: '으로',
 23: '로',
 24: '점',
 25: '않다',
 26: '에서',
 27: '나오다',
 28: '과',
 29: '연기',
 30: '만들다',
 31: '평점',
 32: '최고',
 33: '안',
 34: '인',
 35: '나',
 36: '내',
 37: '그',
 38: '못',
 39: '스토리',
 40: '사람',
 41: '드라마',
 42: '왜',
 43: '보고',
 44: '게',
 45: '고',
 46: '감동',
 47: '생각',
 48: '이렇다',
 49: '아깝다',
 50: '감독',
 51: '때',
 52: '말',
 53: 'ㅋㅋ',
 54: '그냥',
 55: '시간',
 56: '배우',
 57: '재미없다',
 58: '더',
 59: '거',
 60: '와',
 61: '재미',
 62: '요',
 63: '내용',
 64: '지루하다',
 65: '재미있다',
 66: '하고',
 67: '중',
 68: '자다',
 69: '네',
 70: '주다',
 71: '까지',
 72: '가다',
 73: '뭐',
 74: '모르다',
 75: '좀',
 76: '쓰레기',
 77: '작품',
 78: '들다',
 79: '알다',
 80: '지',
 81: '하나',
 82: '싶다',
 83: '사랑',
 84: '이건',
 85: '그렇다',
 86: '볼',
 87: 'ㅋ',
 88: '정도',
 89: '다시',
 90: '잘',
 91: '이렇게',
 92: '마지막',
 93: '액션',


In [47]:
for vl in train_input[0][:4] : 
    print(tmp_dict[vl])

더빙
진짜
짜증나다
목소리


In [48]:
for vl in train_input[1] : 
    print(tmp_dict[vl])

초딩
영화
줄
오버
연기
조차
가볍다
않다


### 모델 구성
- tensorflow.keras.Model 상속 받아서
    - 사용자 정의 모델 클래스 구현

In [49]:
prepro_configs.keys() # 어휘사전 load한 dict 변수

dict_keys(['vocab', 'vocab_size'])

In [50]:
vocab_size = prepro_configs['vocab_size']  # 총 단어수
embbeding_size= 128
vocab_size

12351

### CNN 모델 구성
- 이미지 사용 : Conv2D - 2차원 이미지를 그대로 학습
- 문장과 같은 1차원의 sequential 데이터 cnn 적용할 때 : Conv1D 층
- Embedding 층 사용 (밀집벡터 생성)
- MaxPooling1D : 입력벡터에서 특정 구간마다 최대값을 골라 벡터 구성
- GlobalMaxPooling1D : 여러 개의 벡터 정보 중 가장 큰 벡터를 골라서 반환
    - 필터가 내놓은 특성맵 중에 제일 큰 특성맵 하나만 고름

In [51]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,Flatten,Embedding

In [52]:
model = Sequential()
model.add(Embedding(vocab_size,embbeding_size))
model.add(tf.keras.layers.Conv1D(100,5,activation='relu'))
model.add(tf.keras.layers.GlobalMaxPooling1D())
model.add(tf.keras.layers.Dropout(rate=0.2))
model.add(tf.keras.layers.Dense(250, activation='relu'))
model.add(tf.keras.layers.Dense(1,activation='sigmoid'))

In [53]:
model.compile(optimizer='adam',
             loss='binary_crossentropy',
             metrics=['accuracy'])

In [54]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, None, 128)         1580928   
                                                                 
 conv1d (Conv1D)             (None, None, 100)         64100     
                                                                 
 global_max_pooling1d (Globa  (None, 100)              0         
 lMaxPooling1D)                                                  
                                                                 
 dropout (Dropout)           (None, 100)               0         
                                                                 
 dense (Dense)               (None, 250)               25250     
                                                                 
 dense_1 (Dense)             (None, 1)                 251       
                                                        

In [55]:
train_input.shape

(10000, 8)

In [56]:
train_label.shape

(10000,)

In [57]:
BATCH_SIZE = 512
NUM_EPOCHS = 10
VALID_SPLIT = 0.1
MAX_LEN = train_input.shape[1]
model_name = 'naver'

### EarlyStopping(min_delta=n)
- 개선된 것으로 간주하기 위한 최소한의 변화량
- ex. min_delta값이 0.01dlrh 50 epoch에 정확도가 0.86 라고 할 때, 51 epoch에서 정확도가 0.8652로 높아졌다.
- 이는 0.0052의 개선이 있었지만, min_delta값인 0.01에 미치지 못하기 때문에 개선되었다고 보지 않고 patience를 확인한 후에 조기종료 할 수도 있다.

In [58]:
# 검증 정확도를 통한 EarlyStopping 기능 및 모델 저장 방식 지정
earlystop_callback = EarlyStopping(monitor='val_acc', 
                                   min_delta=0.0001,
                                   patience=2)

checkpoint_path = DATA_OUT + model_name +'weights.h5'
checkpoint_dir = os.path.dirname(checkpoint_path)

In [59]:
checkpoint_path

'data_out/naverweights.h5'

In [60]:
if os.path.exists(checkpoint_dir):
    print("{} -- Folder already exists \n".format(checkpoint_dir))
else:
    os.makedirs(checkpoint_dir, exist_ok=True)
    print("{} -- Folder create complete \n".format(checkpoint_dir))

data_out -- Folder already exists 



In [61]:
cp_callback = ModelCheckpoint(
    checkpoint_path, monitor = 'val_acc', verbose=1, save_best_only = True,
    save_weights_only=True
)

In [62]:
history = model.fit(train_input,
                    train_label,
                    batch_size=BATCH_SIZE,
                    epochs = NUM_EPOCHS,
                    validation_split=VALID_SPLIT,  # 검증 비율 전달
                    callbacks=[earlystop_callback,cp_callback]
                   )

Epoch 1/10
18/18 [==============================] - 7s 53ms/step - loss: 0.6876 - accuracy: 0.5639 - val_loss: 0.6711 - val_accuracy: 0.7030
Epoch 2/10
18/18 [==============================] - 0s 9ms/step - loss: 0.5714 - accuracy: 0.7894 - val_loss: 0.5245 - val_accuracy: 0.7490
Epoch 3/10
18/18 [==============================] - 0s 7ms/step - loss: 0.3268 - accuracy: 0.8672 - val_loss: 0.5540 - val_accuracy: 0.7450
Epoch 4/10
18/18 [==============================] - 0s 6ms/step - loss: 0.1925 - accuracy: 0.9313 - val_loss: 0.6050 - val_accuracy: 0.7450
Epoch 5/10
18/18 [==============================] - 0s 7ms/step - loss: 0.1169 - accuracy: 0.9626 - val_loss: 0.6998 - val_accuracy: 0.7440
Epoch 6/10
18/18 [==============================] - 0s 7ms/step - loss: 0.0738 - accuracy: 0.9767 - val_loss: 0.8078 - val_accuracy: 0.7470
Epoch 7/10
18/18 [==============================] - 0s 7ms/step - loss: 0.0515 - accuracy: 0.9833 - val_loss: 0.9195 - val_accuracy: 0.7400
Epoch 8/10
18/18 [=

In [63]:
TEST_INPUT_DATA = 'nsmc_test_input.npy'
TEST_LABEL_DATA = 'nsmc_test_label.npy'

test_input = np.load(open(DATA_PATH +TEST_INPUT_DATA,'rb' ))
test_label = np.load(open(DATA_PATH +TEST_LABEL_DATA,'rb' ))

In [64]:
test_input.shape
test_label.shape

(10000,)

In [65]:
model.evaluate(test_input, test_label)

313/313 [==============================] - 1s 2ms/step - loss: 1.1657 - accuracy: 0.7407


[1.165748953819275, 0.7407000064849854]

In [66]:
# 최종 모델 저장
model.save('review.h5')

### 예측하기
- 저장된 모델을 활용해서 예측 프로세스 생성

In [67]:
import tensorflow as tf
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras import layers
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import json
from tqdm import tqdm

In [68]:
# 모델 불러오기
model = tf.keras.models.load_model('review.h5')

### 학습 후 완성된 모델을 저장했음
- 이 모델을 이용해 새로 입력되는 문장에 대해 긍/부정 판단
- **새로 입력되는 문장도 학습데이터와 동일하게 전처리 되어야 함**
    - 학습데이터에서 사용한 어휘사전을 그대로 사용해야 함

In [69]:
from konlpy.tag import Okt
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import Tokenizer

okt = Okt()
tokenizer  = Tokenizer()

In [70]:
# 학습 시 사용한 어휘사전 불러오기
DATA_CONFIGS = 'cleandata/data_configs.json'
prepro_configs = json.load(open(DATA_CONFIGS, 'r'))
word_vocab = prepro_configs['vocab']

In [71]:
# tokenizer 구성 - 학습 시 생성해 놓은 어휘사전을 이용해서 재구성
tokenizer.fit_on_texts(word_vocab)

In [72]:
# 나머지 전처리 진행
MAX_LENGTH = 8
sentence = input('감성 분석할 문장을 입력해 주세요 : ')

# 새로 입력된 문장을 전처리
sentence = re.sub(r'[^ㄱ-ㅎㅏ-ㅣ가-힣\\s ]','', sentence)
stopwords = ['은','는','이','가','하','아','것','들','의','있','되','수','보','주','등','한'] # 불용어 추가할 것이 있으면 이곳에 추가
sentence = okt.morphs(sentence, stem=True)  # 형태소 분석 토큰화
sentence = [word for word in sentence if not word in stopwords]  # 불용어 제거
print(sentence)

vector = tokenizer.texts_to_sequences(sentence)
pad_new = pad_sequences([vector],maxlen=MAX_LENGTH,padding='post')
print(pad_new)
model=tf.keras.models.load_model('review.h5')
pred = model.predict(pad_new)
print(pred)
pred=float(pred)

if(pred>0.5) :
    print("{:.2f}%확률로 긍정 리뷰 입니다.\n".format(pred*100))
else : 
    print("{:.2f}%확률로 부정 리뷰 입니다.\n".format((1-pred)*100))

감성 분석할 문장을 입력해 주세요 : 너무 재밌어요!
['너무', '재밌다']
[[[13]
  [15]
  [ 0]
  [ 0]
  [ 0]
  [ 0]
  [ 0]
  [ 0]]]
[[0.98566794]]
98.57%확률로 긍정 리뷰 입니다.

